<a href="https://colab.research.google.com/github/vaishnavivyasam/Vaishnavi_INFO5731_Fall2021/blob/main/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(20 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [Apple iPhone 11](https://www.amazon.com/Apple-iPhone-11-64GB-Unlocked/dp/B07ZPKF8RG/ref=sr_1_13?dchild=1&keywords=iphone+12&qid=1631721363&sr=8-13) on amazon.

(2) Collect all User Reviews of the film [Shang-Chi and the Legend of the Ten Rings](https://www.imdb.com/title/tt9376612/reviews?ref_=tt_sa_3) from IMDB.

(3) Collect all the abstracts research papers by using the query [smart health](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 10,000 tweets by using hashtag ["#blacklivesmatter"](https://twitter.com/hashtag/blacklivesmatter) from Twitter with the following requirements: Location (Texas), Time frame (2021-01-01 to 2021-09-01). 


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

iterations = 100 #no.of iterations

def get_data(itr):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://citeseerx.ist.psu.edu/search?q=smart+health+research+paper'+str(itr), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'result'}): # artcile window
        all1=[]
        #print(d)

        title = d.find('a') # to get title
        if title is not None:
            tt = title.text.strip() # to remove the uncessary spaces from text
            all1.append(tt)
        else:
            all1.append('Unknown') # returns unknows if there is no title

        abstract = d.find('div', attrs={'class':'snippet'}) # to get abstract
        if abstract is not None:
            ab = abstract.text.strip()
            all1.append(ab)
        else:
            all1.append('Not Available')


       
        
        alls.append(all1)

  
    return alls

results = []
for i in range(1, iterations+1):
    results.append(get_data(i+10))
# as the output will be a list, in order to pass it to the DataFrame it needs to be flattened
flatten = lambda a: [item for sublist in a for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Title','Abstract'])
df.shape

df.to_csv('citeseerX.csv')


In [6]:
import pandas as pd
df = pd.read_csv("citeseerX.csv")
df.head()

,Unnamed: 0,Title,Abstract
0,0,The Determinants of Credit Spread Changes.,"""... ABSTRACT Using dealer's quotes and trans..."
1,1,Smart Sensing Technology: Opportunities and Ch...,"""... ABSTRACT: “Smart ” sensors with embedded ..."
2,2,Structural health monitoring using smart sensors,"""... of excellence in research and education t..."
3,3,Whom You Know Matters: Venture Capital Network...,"""... Abstract Many financial markets are char..."
4,4,A Survey on Wearable Sensor-Based Systems for ...,"""... Abstract—The design and development of we..."


# **Question 2**

(30 points) Here is a [legal case](https://github.com/unt-iialab/info5731-fall2021/blob/main/assignments/01-05-1%20%20Adams%20v%20Tanner.txt) we collected from westlaw, please follow the steps we mentioned in lesson 5 to clean the data:



(1) Basic feature extraction using text data

*   Number of sentences
*   Number of words
*   Number of characters
*   Average word length
*   Number of stopwords
*   Number of special characters
*   Number of numerics
*   Number of uppercase words

(2) Basic Text Pre-processing of text data

*   Lower casing
*   Punctuation removal
*   Stopwords removal
*   Frequent words removal
*   Rare words removal
*   Spelling correction
*   Tokenization
*   Stemming
*   Lemmatization

(3) Save all the **clean sentences** to a **csv file** (one column, each raw is a sentence) after finishing all the steps above.


(4) Advance Text Processing

*   Calculate the term frequency of all the terms.
*   Print out top 10 1-gram, top 10 2-grams, and top 10 3-grams terms as features.


In [ ]:
# 2.1 Basic feature extraction using text data

from google.colab import files
uploaded = files.upload()

file_name = "westlaw.txt"
uploaded[file_name].decode("utf-8")

uploaded[file_name].decode("utf-8").split("\r\n")

data = uploaded[file_name].decode("utf-8").split("\r\n")

for i in range(len(data)):
  data[i] = data[i].split(",")

print(data)





Saving westlaw.txt to westlaw.txt
[['5 Ala. 740'], ['Supreme Court of Alabama.'], ['ADAMS'], ['v.'], ['TANNER AND HORTON.'], ['June Term', ' 1843.'], ['Synopsis'], ['WRIT of Error to the Circuit Court of Sumter.'], [' '], [''], [''], ['West Headnotes (2)'], [''], [''], ['[1]'], ['Chattel Mortgages'], ['Crops'], ['A growing crop has such an existence as to be the subject-matter of a mortgage or other contract which passes an interest to vest in possession', ' either immediately or at a future time.'], ['4 Cases that cite this headnote'], [''], ['[2]'], ['Creditors’ Remedies'], ['Lien and Priority'], ['Under St.1821', ' prohibiting a levy on a crop until it has been gathered', ' no lien attaches in favor of a fi. fa. on a growing crop', ' nor does such lien attach until after the crop has been gathered.'], ['5 Cases that cite this headnote'], [''], ['**1 This was a trial of the right of property under the statute. In November', ' 1840', ' an execution issued from the circuit court of Sum

In [ ]:
#no.of sentences, words amd characters
file = open("westlaw.txt", "r")

number_of_sentences = 0
number_of_words = 0
number_of_characters = 0
count1=0
count2=0
for line in file:
  line = line.strip("\n")
  sentence = line.count(".")


  words = line.split()
  #sentences += line.count('.')
  number_of_sentences += 1
  number_of_words += len(words)
  number_of_characters += len(line)



file.close()

print("Number of Sentences:", number_of_sentences, "\nNumber of Words:", number_of_words, "\nNumber of Characters:", number_of_characters)

Number of Sentences: 158 
Number of Words: 3707 
Number of Characters: 20295


In [ ]:
#avg word count

newfile=input('Enter file name: ')
f=open(newfile,'r')
count1=0
count2=0
for line in f:
    count1+=1
    words=line.rstrip().split()
for word in words:
    count2+=1
average=count1/count2
print('Average word length: ',average)

Enter file name: westlaw.txt
Average word length:  19.75


In [ ]:
#no.of stopwords

import nltk
nltk.download('stopwords')
stop = stopwords.words('english')
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'bo

In [ ]:
#Number of Numericals

import re

digit_count=0
number_count = 0
numbers = []
#count=0

with open ("westlaw.txt") as f:
  for line in f.readlines():
    sub_strs = line.rstrip().split("-")
    for i in range(0, len(sub_strs)):
      file_words = re.split(r"[a-zA-Z\W]",sub_strs[i])
      for word in file_words:
        if word.isdigit():
          number_count += 1
          if i >=1 and sub_strs[i].startswith(word):
            numbers.append("-")
            numbers.append(word)
          else:
            numbers.append(word)


print("Number of Numericals:",number_count)
print("numbers:",numbers)

Number of Numericals: 156
numbers: ['5', '740', '1843', '2', '1', '4', '2', '1821', '5', '1', '1840', '80', '-', '100', '1839', '741', '7', '1840', '1840', '-', '1', '1840', '2', '-', '1', '361', '-', '1', '307', '6', '604', '-', '1', '2', '418', '422', '7', '34', '41', '167', '742', '3', '112', '207', '3', '338', '424', '5', '26', '13', '235', '8', '693', '4', '1821', '167', '2', '2', '216', '3', '66', '4', '130', '29', '2', '241', '-', '2', '332', '-', '2', '422', '9', '112', '743', '9', '39', '1840', '744', '5', '182', '3', '368', '1', '397', '6', '604', '1821', '167', '745', '4', '746', '4', '210', '46', '747', '5', '5', '740', '1843', '284', '2019', '9', '1', '55', '266', '271', '1876', '2', '47', '362', '376', '1872', '3', '45', '329', '334', '1871', '4', '31', '526', '527', '1858', '5', '21', '333', '335', '1852', '6', '8', '145', '147', '1857', '7', '65', '256', '258', '1880', '8', '4', '913', '914', '1887', '9', '103', '464', '1936', '3', '1', '9', '39', '1828', '2', '2', '5',

In [ ]:
#Number of uppercase

def count_capital_words(westlaw):                                               
    count = 0                                                                    
    with open(westlaw, 'r') as fp:                                              
        for line in fp:                                                          
            for word in filter(None, line.split()):                              
                if word[0].isupper():                                            
                    count += 1                                                   
    return count


print("Number of Uppercase letter:",count_capital_words('westlaw.txt')) 

Number of Uppercase letter: 462


In [ ]:
#2.2 Basic Text Pre-processing of text data
#Lower casing
with open('westlaw.txt', 'r') as fileinput:
   for line in fileinput:
       line = line.lower()
       print(line)


5 ala. 740

supreme court of alabama.

adams

v.

tanner and horton.

june term, 1843.

synopsis

writ of error to the circuit court of sumter.

 





west headnotes (2)





[1]

chattel mortgages

crops

a growing crop has such an existence as to be the subject-matter of a mortgage or other contract which passes an interest to vest in possession, either immediately or at a future time.

4 cases that cite this headnote



[2]

creditors’ remedies

lien and priority

under st.1821, prohibiting a levy on a crop until it has been gathered, no lien attaches in favor of a fi. fa. on a growing crop, nor does such lien attach until after the crop has been gathered.

5 cases that cite this headnote



**1 this was a trial of the right of property under the statute. in november, 1840, an execution issued from the circuit court of sumter, at the suit of the plaintiff in error, requiring the sheriff of that county, to make of the goods, &c., of allen harrison and others, the sum of thirty-seven

In [ ]:
#Punctuation removal
text_file = open('westlaw.txt', 'r')
text = text_file.read()

# initializing punctuations string

punc = '''!()-[]{};:,.'"?@#$%^&*'''

# Using loop + punctuation string
for ele in text:
    if ele in punc:
        text = text.replace(ele, "")
 
# printing result
print(text)


5 Ala 740
Supreme Court of Alabama
ADAMS
v
TANNER AND HORTON
June Term 1843
Synopsis
WRIT of Error to the Circuit Court of Sumter
 


West Headnotes 2


1
Chattel Mortgages
Crops
A growing crop has such an existence as to be the subjectmatter of a mortgage or other contract which passes an interest to vest in possession either immediately or at a future time
4 Cases that cite this headnote

2
Creditors’ Remedies
Lien and Priority
Under St1821 prohibiting a levy on a crop until it has been gathered no lien attaches in favor of a fi fa on a growing crop nor does such lien attach until after the crop has been gathered
5 Cases that cite this headnote

1 This was a trial of the right of property under the statute In November 1840 an execution issued from the circuit court of Sumter at the suit of the plaintiff in error requiring the sheriff of that county to make of the goods c of Allen Harrison and others the sum of thirtyseven hundred and seventyseven 80100 dollars besides costs This exec

In [ ]:
#stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
# Get nltk stopword list into a set.
stop_words = set(stopwords.words('english'))
 
# Open and read in a text file.
txt_file = open("westlaw.txt")
txt_line = txt_file.read()
txt_words = txt_line.split()
 
# stopwords found counter.
sw_found = 0
 
# If each word checked is not in stopwords list,
# then append word to a new text file.
for check_word in txt_words:
    if not check_word.lower() in stop_words:
        # Not found on stopword list, so append.
        appendFile = open('stopwords-removed.txt','a')
        appendFile.write(" "+check_word)
        appendFile.close()
    else:
        # It's on the stopword list
        sw_found +=1
        print(check_word)
 
print(sw_found,"stop words found and removed")
print("Saved as 'stopwords-removed.txt' ")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
of
AND
of
to
the
of
A
has
such
an
as
to
be
the
of
a
or
other
which
an
to
in
or
at
a
that
this
and
Under
a
on
a
until
it
has
been
no
in
of
a
on
a
nor
does
such
until
after
the
has
been
that
this
This
was
a
of
the
of
under
the
In
an
from
the
of
at
the
of
the
in
the
of
that
to
of
the
of
and
the
of
and
This
was
on
of
as
the
of
which
was
and
a
to
the
An
was
up
to
the
of
the
of
the
to
the
and
to
a
On
the
a
of
was
at
the
of
the
The
that
he
his
in
that
an
was
on
the
and
that
and
up
to
the
was
that
the
on
was
on
the
of
and
by
the
in
his
It
was
by
the
by
the
of
a
that
on
the
of
in
that
the
were
as
for
of
and
were
then
to
an
to
of
and
to
the
all
his
of
of
and
and
himself
to
up
his
to
the
of
the
at
any
to
them
from
as
his
if
the
and
was
he
to
the
in
The
from
they
about
the
of
with
them
or
and
of
the
and
and
with
the
and
the
it
for
and
when
it
was
in
a
in
The
then
by
that
the
of
the
while


In [ ]:
#Removal of Frequent words

s = open('stopwords-removed.txt','r').read()  # Open the input file

# Program will count the characters in text file
num_chars = len(s)

# Program will count the lines in the text file
num_lines = s.count('\n')

# Program will call split with no arguments
words = s.split()
d = {}
for w in words:
    if w in d:
        d[w] += 1
    else:
        d[w] = 1

num_words = sum(d[w] for w in d)

lst = [(d[w],w) for w in d]
lst.sort()
lst.reverse()

# Program assumes user has downloaded an imported stopwords from NLTK
from nltk.corpus import stopwords # Import the stop word list
from nltk.tokenize import wordpunct_tokenize

stop_words = set(stopwords.words('english')) # creating a set makes the searching faster
print ([word for word in lst if word not in stop_words])

i = 1
for count, word in lst[:10]:
    print('%2s. %4s %s' %(i,count,word))
    i+= 1

[(36, 'execution'), (29, 'crop'), (23, 'levy'), (22, 'v.'), (20, 'lien'), (17, 'right'), (15, 'crop,'), (15, 'Rep.'), (14, 'contract'), (14, 'Ala.'), (13, 'growing'), (13, 'claimants'), (12, 'defendant'), (11, '—'), (11, 'upon'), (11, 'Court'), (11, 'Case'), (10, 'time'), (10, 'law,'), (9, 'possession'), (9, 'gathered,'), (9, 'execution,'), (8, 'would'), (8, 'plaintiff'), (8, 'may'), (8, 'act'), (8, ','), (7, 'taken'), (7, 'subject'), (7, 'sale'), (7, 'property'), (7, 'made'), (7, 'levied'), (7, 'court'), (7, 'cotton'), (7, 'common'), (7, 'Term'), (7, 'Johns.'), (7, 'Cited'), (6, 'statute'), (6, 'sheriff'), (6, 'shall'), (6, 'planted'), (6, 'debtor,'), (6, 'could'), (6, 'case,'), (6, 'Tried'), (6, 'Hon.'), (6, 'Harrison,'), (6, 'Circuit'), (6, '5'), (5, 'without'), (5, 'whether'), (5, 'judgment'), (5, 'interest'), (5, 'give'), (5, 'fieri'), (5, 'claimants,'), (5, 'attached'), (5, 'APPEAL'), (5, 'A.'), (5, '2'), (5, '1840,'), (5, '&'), (4, 'took'), (4, 'sold'), (4, 'sell'), (4, 'sale,')

In [ ]:
#Rare words removal 

text_file = open('stopwords-removed.txt', 'r')
text = text_file.read()

#cleaning
text = text.lower()
words = text.split()
words = [word.strip('.,!;''()[]&') for word in words]
words = [word.replace("'s", '') for word in words]

#finding unique
unique = []
for word in words:
    if word not in unique:
        unique.append(word)

#sort
#unique.sort()

#print
print(unique)



['5', 'ala', '740', 'supreme', 'court', 'alabama', 'adams', 'v', 'tanner', 'horton', 'june', 'term', '1843', 'synopsis', 'writ', 'error', 'circuit', 'sumter', 'west', 'headnotes', '2', '1', 'chattel', 'mortgages', 'crops', 'growing', 'crop', 'existence', 'subject-matter', 'mortgage', 'contract', 'passes', 'interest', 'vest', 'possession', 'either', 'immediately', 'future', 'time', '4', 'cases', 'cite', 'headnote', 'creditors’', 'remedies', 'lien', 'priority', 'st.1821', 'prohibiting', 'levy', 'gathered', 'attaches', 'favor', 'fi', 'fa', 'attach', '**1', 'trial', 'right', 'property', 'statute', 'november', '1840', 'execution', 'issued', 'suit', 'plaintiff', 'requiring', 'sheriff', 'county', 'make', 'goods', 'c', 'allen', 'harrison', 'others', 'sum', 'thirty-seven', 'hundred', 'seventy-seven', '80-100', 'dollars', 'besides', 'costs', 'levied', 'thirty', 'bales', 'cotton', 'claimed', 'bond', 'given', 'try', 'issue', 'made', 'question', 'liability', 'plaintiff’s', 'submitted', 'jury', 'bil

In [ ]:
pip install textblob

In [ ]:
#Spelling correction

from textblob import TextBlob

with open("stopwords-removed.txt", "r") as f:       
    text = f.read()
    # Making our first textblob
    textBlb = TextBlob(text)
    # Correcting the text   
    textCorrected = textBlb.correct()
    print(textCorrected)

5 La. 740 
Supreme Court Alabama. 
ADAMS 
v. 
TANNER HORTON. 
June Term, 1843. 
Synopsis 
WRIT Error Circuit Court Sumter. 
West Headnotes (2) 

[1] 
Chattel Mortgages 
Drops 
growing crop existence subject-matter mortgage contract passes interest vest possession, either immediately future time. 
4 Cases cite headnote 

[2] 
Creditors’ Remedies 
Mien Priority 
It.1821, prohibiting levy crop gathered, line attaches favor i. a. growing crop, line attach crop gathered. 
5 Cases cite headnote 

**1 trial right property statute. November, 1840, execution issued circuit court Sumter, suit plaintiff error, requiring sheriff county, make goods, &c., Ellen Harrison others, sum thirty-seven hundred seventy-seven 80-100 dollars, besides costs. execution levied thirty bales cotton, property Ellen Harrison, claimed, bond given try right. issue made try question liability cotton plaintiff’s execution, submitted jury. trial, bill exceptions sealed instance plaintiff. plaintiff proved recovered judgme

In [ ]:
#Tokenization
import nltk
nltk.download('punkt')
fileObj = open('stopwords-removed.txt', 'r')
text = fileObj.read()
tokens = nltk.sent_tokenize(text)
print(tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
['5 Ala. 740 \nSupreme Court Alabama.', 'ADAMS \nv. \nTANNER HORTON.', 'June Term, 1843.', 'Synopsis \nWRIT Error Circuit Court Sumter.', 'West Headnotes (2) \n\n[1] \nChattel Mortgages \nCrops \ngrowing crop existence subject-matter mortgage contract passes interest vest possession, either immediately future time.', '4 Cases cite headnote \n\n[2] \nCreditors’ Remedies \nLien Priority \nSt.1821, prohibiting levy crop gathered, lien attaches favor fi.', 'fa.', 'growing crop, lien attach crop gathered.', '5 Cases cite headnote \n\n**1 trial right property statute.', 'November, 1840, execution issued circuit court Sumter, suit plaintiff error, requiring sheriff county, make goods, &c., Allen Harrison others, sum thirty-seven hundred seventy-seven 80-100 dollars, besides costs.', 'execution levied thirty bales cotton, property Allen Harrison, claimed, bond given try right.', 'issue mad

In [ ]:
#Stemming
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
#sentence="Hello Guru99, You have to build a very good site and I love visiting your site."
with open("stopwords-removed.txt", "r") as f:       
    text = f.read()
words = word_tokenize(text)
ps = PorterStemmer()
for w in words:
	rootWord=ps.stem(w)
	print(rootWord)

5
ala.
740
suprem
court
alabama
.
adam
v.
tanner
horton
.
june
term
,
1843
.
synopsi
writ
error
circuit
court
sumter
.
west
headnot
(
2
)
[
1
]
chattel
mortgag
crop
grow
crop
exist
subject-matt
mortgag
contract
pass
interest
vest
possess
,
either
immedi
futur
time
.
4
case
cite
headnot
[
2
]
creditor
’
remedi
lien
prioriti
st.1821
,
prohibit
levi
crop
gather
,
lien
attach
favor
fi
.
fa
.
grow
crop
,
lien
attach
crop
gather
.
5
case
cite
headnot
**1
trial
right
properti
statut
.
novemb
,
1840
,
execut
issu
circuit
court
sumter
,
suit
plaintiff
error
,
requir
sheriff
counti
,
make
good
,
&
c.
,
allen
harrison
other
,
sum
thirty-seven
hundr
seventy-seven
80-100
dollar
,
besid
cost
.
execut
levi
thirti
bale
cotton
,
properti
allen
harrison
,
claim
,
bond
given
tri
right
.
issu
made
tri
question
liabil
cotton
plaintiff
’
s
execut
,
submit
juri
.
trial
,
bill
except
seal
instanc
plaintiff
.
plaintiff
prove
recov
judgment
octob
,
1839
;
execut
*741
issu
thereon
7th
nov.
thereaft
,
alia
pluri


In [ ]:
#Lemmatization
import nltk
nltk.download('wordnet')
from nltk.stem import 	WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
#text = "studies studying cries cry"
with open("stopwords-removed.txt", "r") as f:       
    text = f.read()
tokenization = nltk.word_tokenize(text)
for w in tokenization:
 print("Lemma for {} is {}".format(w, wordnet_lemmatizer.lemmatize(w)))  

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Lemma for 5 is 5
Lemma for Ala. is Ala.
Lemma for 740 is 740
Lemma for Supreme is Supreme
Lemma for Court is Court
Lemma for Alabama is Alabama
Lemma for . is .
Lemma for ADAMS is ADAMS
Lemma for v. is v.
Lemma for TANNER is TANNER
Lemma for HORTON is HORTON
Lemma for . is .
Lemma for June is June
Lemma for Term is Term
Lemma for , is ,
Lemma for 1843 is 1843
Lemma for . is .
Lemma for Synopsis is Synopsis
Lemma for WRIT is WRIT
Lemma for Error is Error
Lemma for Circuit is Circuit
Lemma for Court is Court
Lemma for Sumter is Sumter
Lemma for . is .
Lemma for West is West
Lemma for Headnotes is Headnotes
Lemma for ( is (
Lemma for 2 is 2
Lemma for ) is )
Lemma for [ is [
Lemma for 1 is 1
Lemma for ] is ]
Lemma for Chattel is Chattel
Lemma for Mortgages is Mortgages
Lemma for Crops is Crops
Lemma for growing is growing
Lemma for crop is crop
Lemma for existence is existence
Lemma f

2.3 Saving all the clean sentences to a csv file 


In [7]:
# importing panda library
import pandas as pd
df = pd.read_fwf('stopwords-removed.txt')
df.to_csv('stopwords-removed.csv')

In [2]:
import pandas as pd
df = pd.read_csv("stopwords-removed.txt", sep="\n")
df

,5 Ala. 740
0,Supreme Court Alabama.
1,ADAMS
2,v.
3,TANNER HORTON.
4,"June Term, 1843."
...,...
136,Negative Treatment
137,Negative Treatment
138,results citation.
139,History


2.4 Advance Text Processing

Calculate the term frequency of all the terms.
Print out top 10 1-gram, top 10 2-grams, and top 10 3-grams terms as features

In [ ]:
#Frequency of all terms
import re
import string
frequency = {}
document_text = open('stopwords-removed.txt', 'r')
text_string = document_text.read().lower()
match_pattern = re.findall(r'\b[a-z]{3,15}\b', text_string)

for word in match_pattern:
    count = frequency.get(word,0)
    frequency[word] = count + 1
    
frequency_list = frequency.keys()

for words in frequency_list:
    print(words, frequency[words])

ala 15
supreme 1
court 20
alabama 1
adams 1
tanner 1
horton 1
june 1
term 10
synopsis 1
writ 4
error 6
circuit 10
sumter 3
west 1
headnotes 1
chattel 3
mortgages 1
crops 3
growing 16
crop 49
existence 2
subject 9
matter 4
mortgage 6
contract 16
passes 1
interest 5
vest 2
possession 13
either 2
immediately 2
future 3
time 12
cases 3
cite 2
headnote 3
creditors 3
remedies 1
lien 25
priority 1
prohibiting 2
levy 25
gathered 19
attaches 4
favor 3
attach 2
trial 3
right 21
property 10
statute 9
november 1
execution 50
issued 3
suit 1
plaintiff 14
requiring 2
sheriff 8
county 3
make 2
goods 1
allen 3
harrison 10
others 1
sum 1
thirty 2
seven 2
hundred 2
seventy 1
dollars 3
besides 1
costs 1
levied 8
bales 2
cotton 14
claimed 2
bond 1
given 2
try 2
issue 3
made 8
question 5
liability 3
submitted 1
jury 4
bill 3
exceptions 3
sealed 1
instance 2
proved 3
recovered 1
judgment 7
october 1
thereon 1
nov 2
thereafter 1
alias 1
pluries 1
fieri 5
facias 6
regularly 1
growed 1
plantation 1
cultivated 

# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
# Parts of speech(POS) Tagging
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize, pos_tag, pos_tag_sents
text = open("stopwords-removed.txt", "r")
text = text.read()
tagged_texts = pos_tag_sents(map(word_tokenize, text))
print(tagged_texts)

nouns=0
verbs=0
adjectives=0
adverbs=0

for i in tagged_texts:
    for j in i:
        if 'NN' in j:
            nouns=nouns+1
        if 'VB' in j:
            verbs+=1
        if 'ADJ' in j:
            adjectives+=1
        if 'ADV' in j:
            adverbs+=1
        
print('nouns:', nouns)
print('verbs:', verbs)
print('adjectives:', adjectives)
print('adverbs:', adverbs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[[('5', 'CD')], [], [('A', 'DT')], [('l', 'NN')], [('a', 'DT')], [('.', '.')], [], [('7', 'CD')], [('4', 'CD')], [('0', 'CD')], [], [], [('S', 'NN')], [('u', 'NN')], [('p', 'NN')], [('r', 'NN')], [('e', 'NN')], [('m', 'NN')], [('e', 'NN')], [], [('C', 'SYM')], [('o', 'NN')], [('u', 'NN')], [('r', 'NN')], [('t', 'NN')], [], [('A', 'DT')], [('l', 'NN')], [('a', 'DT')], [('b', 'NN')], [('a', 'DT')], [('m', 'NN')], [('a', 'DT')], [('.', '.')], [], [], [('A', 'DT')], [('D', 'NN')], [('A', 'DT')], [('M', 'NN')], [('S', 'NN')], [], [], [('v', 'NN')], [('.', '.')], [], [], [('T', 'NN')], [('A', 'DT')], [('N', 'NN')], [('N', 'NN')], [('E', 'NN')], [('R', 'NN')], [], [('H', 'NN')], [('O', 'NN')], [('R', 'NN')], [('T', 'NN')], [('O', 

In [ ]:
#Constituency Parsing and Dependency Parsing
import spacy
nlp=spacy.load('en_core_web_sm')
text = open("stopwords-removed.txt", "r")
text = text.read()
for token in nlp(text):
 print(token.text,'=>',token.dep_,'=>',token.head.text)

5 => ROOT => 5
Ala. => nmod => ADAMS
740 => nummod => Ala.

 =>  => 740
Supreme => compound => Court
Court => compound => Alabama
Alabama => appos => Ala.
. => punct => Ala.

 =>  => .
ADAMS => ROOT => ADAMS

 =>  => ADAMS
v. => ROOT => v.

 =>  => v.
TANNER => compound => HORTON
HORTON => ROOT => HORTON
. => punct => HORTON

 =>  => .
June => compound => Term
Term => ROOT => Term
, => punct => Term
1843 => appos => Term
. => punct => Term

 =>  => .
Synopsis => ROOT => Synopsis

 =>  => Synopsis
WRIT => compound => Sumter
Error => compound => Court
Circuit => compound => Court
Court => compound => Sumter
Sumter => ROOT => Sumter
. => punct => Sumter

 =>  => .
West => compound => Headnotes
Headnotes => ROOT => Headnotes
( => punct => 2
2 => punct => 1
) => punct => 2


 =>  => )
[ => punct => 1
1 => appos => Headnotes
] => punct => 1

 =>  => ]
Chattel => compound => Mortgages
Mortgages => ROOT => Mortgages

 =>  => Mortgages
Crops => nmod => contract

 =>  => Crops
growing => amod =>

In [ ]:
! pip install benepar

     |████████████████████████████████| 3.3 MB 15.5 MB/s 
     |████████████████████████████████| 2.9 MB 61.3 MB/s 
     |████████████████████████████████| 1.2 MB 56.3 MB/s 
     |████████████████████████████████| 636 kB 74.1 MB/s 
     |████████████████████████████████| 895 kB 72.3 MB/s 
     |████████████████████████████████| 52 kB 1.8 MB/s 
  Created wheel for benepar: filename=benepar-0.2.0-py3-none-any.whl size=37647 sha256=d8737ffcdcd49dac4229f842a529e7f0c1afed4dd59d419e17ee8365893e4577
  Stored in directory: /root/.cache/pip/wheels/dc/6f/a3/4d27ce92766bdedd2cbbbedb8857fb7a53534331191cda4994
Successfully built benepar
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4 MB 1.2 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051302 sha256=0571e9b77b803b1ca9d3248334d7bea287f842d382fbab968c26a66d5e22e1b8
  Stored in directory: /tmp/pip-ephem-wheel-cache-pdhnsv8v/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
import benepar
benepar.download('benepar_en3')

[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


True

In [ ]:
from benepar.spacy_plugin import BeneparComponent

nlp = spacy.load('en')
nlp.add_pipe(BeneparComponent('benepar_en3'))

text = open("stopwords-removed.txt", "r")
text = text.read()

for i in range(0,99):
  list (nlp(text).sents)[0]._.parse_string

In [ ]:
#Named Entity Recognition
import spacy
  
nlp = spacy.load('en_core_web_sm')
  
text = open("stopwords-removed.txt", "r")
text = text.read()
  
doc = nlp(text)
  
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

5 0 1 CARDINAL
Supreme Court 12 25 ORG
Alabama 26 33 GPE
June Term, 63 73 DATE
1843 74 78 DATE
West Headnotes 125 139 GPE
2 141 142 CARDINAL
1 147 148 CARDINAL
4 299 300 CARDINAL
2 324 325 CARDINAL
lien 405 409 PERSON
lien attach 447 458 PERSON
5 475 476 CARDINAL
1 501 502 CARDINAL
November, 1840 533 547 DATE
Allen Harrison 653 667 PERSON
thirty-seven hundred 680 700 QUANTITY
seventy-seven 701 714 CARDINAL
thirty bales 763 775 QUANTITY
Allen Harrison 793 807 PERSON
October, 1839 1006 1019 DATE
741 1032 1035 CARDINAL
thereon 7th Nov. 1043 1059 ORG
alias pluries fieri facias’ 1072 1099 ORG
Harrison 1166 1174 ORG
Harrison 1249 1257 ORG
twenty-second May, 1840 1259 1282 DATE
Burton & Harrison 1328 1345 ORG
fourteen thousand dollars 1398 1423 MONEY
one hundred twenty acres 1486 1510 QUANTITY
Gainesville 1648 1659 GPE
Tennessee 1676 1685 GPE
first September, 1840 1703 1724 DATE
three four 1735 1745 CARDINAL
Gainesville 1871 1882 GPE
Harrison 1901 1909 PERSON
Harrison 2091 2099 ORG
lien fieri

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

In [ ]:
**Explaination**

In Dependency parsing, I Used Spacy library, and parsed the text and analyzed the sentence. 
I Used for loop to iterate all the text data and analyzed all the sentences in the file. 

The consituency parsing is breaking down the sentences furthermore to analyze the grammar like sub-phrases.



# **Question 4**

(20 points) Python Regular Expression.

(1) Write a Python program to remove leading zeros from an IP address.

ip = "260.08.094.109"


(2) Write a Python Program to extract all the years from the following sentence.

sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."

In [ ]:
# part 1
def removeZeros(ip):

	new_ip = ".".join([str(int(i)) for i in ip.split(".")]) # this will split the ip address by'.' and remove the zeros in the lead
	return new_ip ;

ip ="260.08.094.109"
print(removeZeros(ip))

260.8.94.109


In [ ]:
#part 2

import re
result=re.findall(r'\d{4}',"The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump.")
print (result)

['2010', '1000', '2010', '2019']
